In [1]:
import configparser
import psycopg2
from sql_queries import copy_table_queries, insert_table_queries

def load_staging_tables(cur, conn):
    for query in copy_table_queries:
        print(query)
        cur.execute(query)
        conn.commit()
        
def insert_tables (cur, conn):
    for query in insert_table_queries:
        print(query)
        cur.execute(query)
        conn.commit()
        
def main():
    config = configparser.ConfigParser()
    config.read('dwh.cfg')
    
    conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
    cur = conn.cursor()
    
    load_staging_tables(cur, conn)
    insert_tables(cur, conn)
    
    conn.close()
    
if __name__ == "__main__":
    main()

 COPY staging_events FROM 's3://udacity-dend/log_data'
                            CREDENTIALS 'aws_iam_role=arn:aws:iam::404050409947:role/dwhRole'
                            REGION 'us-west-2'
                            COMPUPDATE OFF STATUPDATE OFF
                            JSON 's3://udacity-dend/log_json_path.json';
                            
 COPY staging_songs FROM 's3://udacity-dend/song_data'
                            CREDENTIALS 'aws_iam_role=arn:aws:iam::404050409947:role/dwhRole'
                            COMPUPDATE OFF region 'us-west-2'
                            JSON 'auto' ;
                            
INSERT INTO songplays (start_time, user_id, level, song_id, artist_id, session_id, location, user_agent)
    SELECT 
        e.ts, 
        e.user_id, 
        e.level, 
        s.song_id, 
        s.artist_id, 
        e.session_id, 
        e.location,
        e.user_agent
    FROM staging_events e
    JOIN staging_songs s
    ON e.artist = s.artist_name
   